# Import data

In [10]:
import pandas as pd
import numpy as np
import warnings
import eventstudy as es
from tqdm import tqdm
import itertools   
import matplotlib.pyplot as plt
import yfinance as yf
warnings.filterwarnings("ignore")

In [11]:
# read data
cn = pd.read_csv("../../1223/data/concentration/concentration/2020_20230814.csv", index_col=False).dropna()
q = pd.read_csv("../../1223/data/concentration/quote/2020_20230814.csv", index_col=False).dropna()
rf = pd.read_csv("../../1223/data/concentration/rf/rf.csv").dropna().rename(columns={'日期':'date'})
df = pd.merge(q, cn, on=['日期', '股號'], how='left')
df = df.groupby('股號').apply(lambda x: x.sort_values('日期')).reset_index(drop=True)

In [12]:
# get marketret data
stock_code = "^TWII"

start_date = "2020-01-01"
end_date = "2023-12-31" 

twii_data = yf.download(stock_code, start=start_date, end=end_date)
twii_data['date'] =  pd.to_datetime(twii_data.index)
twii_data['ret'] = twii_data['Adj Close'].pct_change()

[*********************100%%**********************]  1 of 1 completed


# Mutate

In [13]:
# mutate col
df = pd.merge(q, cn, on=['日期', '股號'], how='left').reset_index(drop=True)
# df = df.groupby('股號').apply(lambda x: x.sort_values('日期')).reset_index(drop=True)


# 量起
df['成交量_1'] = df.groupby('股號', as_index=False)['成交量'].shift(-1)

def divide_two_cols(df_sub):
    df_sub['volume_delta_1'] = df_sub['成交量_1'] / df_sub['成交量']
    return df_sub

df = df.groupby('股號', as_index=False).apply(divide_two_cols)


# 價揚
# shift 1 假設是成交量出現異常後的下一根進場
df['ret'] = df.groupby('股號')['收盤價'].pct_change()
df['ret_2'] = df.groupby('股號')['ret'].shift(-1)
df = df.dropna()

# Prepare Input

In [14]:
def create_ret_df(df):

    uq_id = df['股號'].unique()
    res_by_ticker = []
    res = None

    for ticker_id in tqdm(uq_id[:len(uq_id)]):
        ticker_df = df[df['股號']==ticker_id].reset_index(drop=True)

        date = ticker_df['日期'].tolist()
        ret = ticker_df['ret'].tolist()

        ticker_ret_df = pd.DataFrame(list(zip(date, ret)),
               columns =['date', f'{ticker_id}'])
        
        res_by_ticker.append(ticker_ret_df)
    for i in tqdm(range(len(res_by_ticker))):
        if i == 0:
            res = res_by_ticker[i]

        else:
            res = pd.merge(res, res_by_ticker[i], how='left', on='date')

    return res


In [15]:
return_df = create_ret_df(df)

100%|██████████| 1350/1350 [00:05<00:00, 262.97it/s]


In [16]:
rf["date"] = pd.to_datetime(rf['date'])
return_df["date"] =  pd.to_datetime(return_df["date"])

main_df_4analysis = pd.merge(return_df, rf, how='left', on='date')
main_df_4analysis = pd.merge(main_df_4analysis, twii_data, how='left', on='date')


In [17]:
uq_id = df['股號'].unique()

for tick in tqdm(uq_id[:]):
    
    # make a copy
    copy = main_df_4analysis.copy()
    
    # ret df     
    copy = copy.dropna(subset=[str(tick), 'ret'])
    returns_df =copy.iloc[:,[main_df_4analysis.columns.get_loc(c) for c in ['date', f'{tick}']]] 
    
    # fama df
    famadf = copy.iloc[:,[copy.columns.get_loc(c) for c in ['date', '收市', 'ret']]].rename(columns={'收市':'RF', 'ret':'MktRt'})
    famadf['Mkt-RF'] = famadf['MktRt'] - famadf['RF']
    famadf['SMB'] = 0
    famadf['HML'] = 0
    
    # merge 
    tmp = pd.merge(returns_df, famadf, how='left', on='date')
    tmp.sort_values(by='date', inplace = True)
    tmp = tmp.dropna()
    
    # sub df
    tmp.iloc[:,[tmp.columns.get_loc(c) for c in ['date', f'{tick}']]].to_csv(f'../df/returns_{tick}.csv')
    tmp.iloc[:,[tmp.columns.get_loc(c) for c in ['date', 'RF', 'MktRt', 'Mkt-RF', 'SMB', 'HML']]].to_csv(f'../df/fama_{tick}.csv')
    


100%|██████████| 1350/1350 [00:26<00:00, 51.08it/s]


# Event Study

In [18]:
uq_id = df['股號'].unique()
final_res = pd.DataFrame(columns=['AR', 'Std. E. AR', 'CAR', 'Std. E. CAR', 'T-stat', 'P-value',
       'evnet_window', 'EventDate', 'symbol'])
err = []

for tick in tqdm(uq_id[:]):
    

    es.Single.import_returns(path=f'../df/returns_{tick}.csv', date_format = '%Y-%m-%d')
    es.Single.import_FamaFrench(path=f'../df/fama_{tick}.csv', date_format = '%Y-%m-%d')

    # df tick
    df_tick = df[df['股號']==tick].reset_index(drop=True)

    # get event date
    vol_increase_id = df_tick[(df_tick['成交量']>500)&(df_tick['volume_delta_1']>2)].index
    event_date = df_tick['日期'][vol_increase_id].to_list()
    # print(tick, event_date)

    for ed in event_date:
        try:
            
            event = es.Single.FamaFrench_3factor(
                security_ticker = str(tick),
                event_date = np.datetime64(str(ed)),
                event_window = (0,+10), 
                estimation_size = 30, # 注意這個
                buffer_size = 30,
                keep_model=False
            )
            tick_res = event.results(decimals=[3,5,3,5,2,2])
            tick_res['evnet_window'] = tick_res.index
            tick_res['EventDate'] =  np.datetime64(ed)
            tick_res['symbol'] = tick
            tick_res = tick_res.reset_index(drop=True)
            # final_res = final_res.append(tick_res, ignore_index=True)
            final_res = pd.concat([final_res, tick_res], ignore_index=True)
        except:
            err.append([tick, ed])
            continue

    # print(final_res)
    

100%|██████████| 1350/1350 [25:05<00:00,  1.12s/it]


# Result Analysis

In [22]:
final_res = final_res.rename(columns={'P-value':'p', 'T-stat':'t'})
final_res['evnet_window'] = final_res['evnet_window'].astype(str).astype(int)
final_res['symbol'] = final_res['symbol'].astype(str)

In [23]:
# final_res.groupby(['symbol', 'EventDate']).filter(lambda x: ((x.p<= 0.05) & (x.evnet_window > 0)).all())
res_analysis = final_res.assign(
    
    # the second position is where to sum
    negative_significant_after_event = np.where((final_res['evnet_window']>0) & (final_res['t']<0), final_res['p']<=0.05, 0),
    positive_significant_after_event = np.where((final_res['evnet_window']>0) & (final_res['t']>0), final_res['p']<=0.05, 0),
    
    
).groupby(['EventDate', 'symbol'], as_index=False).agg({'negative_significant_after_event':sum, 'positive_significant_after_event':sum})

In [30]:
WINDOW_SIZE = 10

## 1. There are more then N volumne increase singal 
N_SIGNAL = 10

## 2. window significant percentage
POSITIVE_SIG_PERCENT = 0.8

In [28]:
res_analysis['pos_sig_per'] = res_analysis['positive_significant_after_event']/WINDOW_SIZE
res_analysis['neg_sig_per'] = res_analysis['negative_significant_after_event']/WINDOW_SIZE

In [31]:
res_analysis[res_analysis['pos_sig_per']>POSITIVE_SIG_PERCENT]

,EventDate,symbol,negative_significant_after_event,positive_significant_after_event,pos_sig_per,neg_sig_per
48,2020-04-10,4102,0,9,0.9,0.0
298,2020-04-14,2392,0,9,0.9,0.0
341,2020-04-14,3013,0,10,1.0,0.0
360,2020-04-14,3163,0,10,1.0,0.0
420,2020-04-14,6245,0,9,0.9,0.0
...,...,...,...,...,...,...
82651,2023-07-27,3234,0,10,1.0,0.0
82663,2023-07-27,3706,0,10,1.0,0.0
82669,2023-07-27,4908,0,10,1.0,0.0
82677,2023-07-27,5388,0,10,1.0,0.0
